In [1]:
import pandas as pd
import numpy as np

import gc
from tqdm import tqdm
import time
import os
import warnings
from datetime import datetime

from chicken_dinner.pubgapi import PUBG
from chicken_dinner.constants import map_dimensions

import pymysql
from sqlalchemy import create_engine

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

warnings.filterwarnings(action = 'ignore')

path = os.getcwd()
data_path = os.path.join(path, 'data')

## Functions for web scrap throwable object

In [6]:
def change_date_format(timestamp):
    timestamp = pd.Timestamp(timestamp).to_pydatetime().replace(tzinfo=None)
    return timestamp

In [7]:
def get_telemetry(match_id, map_name):

    current_match = pubg.match(match_id)
    telemetry = current_match.get_telemetry()

    if map_name == 'Tiger_Main':
        mapx, mapy = map_dimensions['Desert_Main']
    else:
        mapx, mapy = map_dimensions[map_name]
        
    return telemetry, mapy

In [8]:
def rename_column_names(column_names):
    if len(column_names.split('.')) > 2:
        return ('_').join(column_names.split('.')[-2:])
    else:
        return column_names.split('.')[-1]

In [13]:
def get_throwable_df(telemetry, mapy):

    throwable_sample_df = pd.DataFrame()
    
    throwables = telemetry.filter_by('log_player_use_throwable')

    # get use throwable items dataframe of a match
    for throwable in throwables:
        if throwable['weapon']['sub_category'] == 'Throwable':
            throwable_sample_df = pd.concat([throwable_sample_df, pd.json_normalize(throwable.to_dict())], axis = 0, ignore_index = True)

    # match table을 참조하는 Foreign key column
    throwable_sample_df['match_id'] = match_id

    # rename columns
    throwable_sample_df.columns = list(map(lambda x: rename_column_names(x), throwable_sample_df.columns.tolist()))
    throwable_sample_df = throwable_sample_df.rename(columns = {'_D': 'log_created_time'})

    # DB에 삽입하기 위해 single string or None이 포함된 list를 string으로 변환
    throwable_sample_df.loc[:, 'zone'] = throwable_sample_df.loc[:, 'zone'].apply(lambda x: x[0] if len(x) > 0 else np.nan)

    # is_game의 0.1 값이 0.1000000001과 같은 값으로 저장되어 있어서 round 처리
    throwable_sample_df.loc[:, 'is_game'] = round(throwable_sample_df.loc[:, 'is_game'], 1)

    # map 시각화를 위해 y axis dimension 변경
    throwable_sample_df.loc[:, 'location_y'] = mapy - throwable_sample_df.loc[:, 'location_y']

    # date format 변경
    throwable_sample_df.loc[:, 'log_created_time'] = throwable_sample_df.loc[:, 'log_created_time'].apply(lambda x: change_date_format(x))
    
    # 불필요한 컬럼 제거
    throwable_sample_df = throwable_sample_df.drop(columns = ['_T', 'attack_id', 'attack_type', 'ranking', 'attached_items', 'category', 'stack_count'], axis = 1)

    return throwable_sample_df

## Get throwable table by match id's from match_data

In [14]:
%%time

# PUBG api authorize
pubg = PUBG(api_key, shard = 'kakao')

throwable_df = pd.DataFrame()
match_json = pd.read_json(os.path.join(data_path, 'match_data.json'))

for idx in tqdm(range(len(match_json))):

    match_id = match_json['data.id'][idx]
    map_name = match_json['data.attributes.mapName'][idx]
    
    telemetry, mapy = get_telemetry(match_id, map_name)
    
    throwable_df = pd.concat([throwable_df, get_throwable_df(telemetry, mapy)], axis = 0, ignore_index = True)
    
print(f'shape of Throwable df: {throwable_df.shape}')

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [03:47<00:00,  4.85s/it]


## Insert into database

In [ ]:
# # local
# user = 'root'
# password = 'mysql'
# host = 'localhost'
# port = 3306
# database = 'pubg'

# gcp
user = ''
password = ''
host = ''
port = 
database = ''

In [ ]:
def insert_data_to_db(data, table_name, connection, if_exist = 'append'):
    
    data.to_sql(index = False,
                name = table_name,
                con = connection,
                if_exists = if_exist,
                method = 'multi')

In [ ]:
%%time

engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}', encoding = 'utf-8')
engine_conn = engine_connect()

insert_data_to_db(damage_df, 'damage', engine_conn)
engine_conn.close()